In [1]:
import re
import pandas as pd
import numpy as np
import time
import os
import Image
import pytesseract
import pylab as pl
import matplotlib.pyplot as plt
from selenium import webdriver
import scipy.stats as ss
from selenium.webdriver.common.keys import Keys
from datetime import datetime
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
driver = webdriver.Firefox()
pagina='http://www.reclameaqui.com.br/ranking/'
driver.get(pagina)

In [49]:
#driver = webdriver.Firefox()
#pagina='http://www.reclameaqui.com.br/ranking/'
#driver.get(pagina)
#time.sleep(5)
#driver.get(driver.find_element_by_xpath('//*[@id="tabela-ranking"]/tbody/tr[14]/td/div/table[1]/tbody/tr/td/a').get_attribute("href"))
#time.sleep(5)
#driver.get(driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div/ul/li/a').get_attribute("href"))
#time.sleep(5)
#text = driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div[2]/ul/li[1]/span[2]').text
#print text
#driver.get(driver.find_element_by_xpath('//*[@id="todo-conteudo-resultado"]/div/div/center/div/ul/li[7]/a').get_attribute("href"))
#time.sleep(5)
#breakCaptha(driver)
#time.sleep(5)
#driver.get('http://www.google.com')
#driver.quit()

In [4]:
#driver.find_element_by_class_name("dados-reclamacao-fanpage").text

In [5]:
def scrapRanking(arg):
    try:
        for i in range(20):
            arg['business'][i]=driver.find_element_by_xpath('//*[@id="tabela-ranking"]/tbody/tr[14]/td/div/table['+str(i+1)+']/tbody/tr/td/a').get_attribute("title")
            arg['href'][i]=driver.find_element_by_xpath('//*[@id="tabela-ranking"]/tbody/tr[14]/td/div/table['+str(i+1)+']/tbody/tr/td/a').get_attribute("href")
    except:
        return arg

In [6]:
ranking=pd.DataFrame(np.zeros((20,2)))
ranking.columns=['business','href']
scrapRanking(ranking)

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
meses = [u'Janeiro', u'Fevereiro', u'Março', u'Abril', u'Maio', u'Junho', u'Julho', u'Agosto', u'Setembro', u'Outubro', u'Novembro', u'Dezembro']

In [8]:
def setDate(txt):
    try:
        for i in range(12):
            txt = txt.replace(meses[i],u'0'+str(i+1))
            txt = txt.replace(u'de', '-')
            match = re.search(r'\d{2} - \d{2} - \d{4} - \d{2}:\d{2}', txt)
        return match, txt
    except:
        print 'Is not a String object'

In [9]:
def getDate(arg):
    match,txt = setDate(arg)
    date = time.strptime(match.group().replace(" ",""), '%d-%m-%Y-%H:%M')
    return date

In [10]:
def getPlace(arg):
    match, arg = setDate(arg)
    arg = arg.replace(match.group(),"")
    m = re.match(r"(\w+)", arg).group(0)
    arg = arg.replace(m,"")
    match = re.search(r', , ', arg)
    arg = arg.replace(match.group(),"")
    return arg

In [11]:
def breakCaptha(wd):
    try:
        oldtab = wd.current_window_handle
        body = wd.find_element_by_tag_name("body")
        body.send_keys(Keys.CONTROL + 't')
        wd.get('http://www.reclameaqui.com.br/indices/lista_reclamacoes/captcha.php')
        #You have to create the folder /captcha
        wd.save_screenshot("../marcim/captcha/screenshot.png")
        wd.find_element_by_tag_name("body").send_keys(Keys.ALT + Keys.NUMPAD1)
        wd.switch_to_window(oldtab)
        captcha = pytesseract.image_to_string(Image.open('../marcim/captcha/screenshot.png'), config="-psm 6")
        cp = driver.find_element_by_name('captcha')
        cp.send_keys(captcha)
        wd.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div[2]/ul/div/div/form/input[2]').click()
    except:
        print "Didn't broke the captcha"

In order to gain performance, we prefer to get a list of URLs in third level of scraping, and then visit them. This already give us O(n³), if we had choose to scrap the fourth level, we would get a O(n⁴) of complexity order.

In [12]:
def thirdLevelScrap(i):
    try:
        subsubtable = pd.DataFrame(np.zeros((20,4)))
        subsubtable.columns=['business','url','date','place']
        for j in range(20):
            subsubtable['business'][j] = ranking['business'][i]
            subsubtable['url'][j] = driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div[2]/ul/li['+str(j+1)+']/h3/a').get_attribute("href")
            text = driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div[2]/ul/li['+str(j+1)+']/span[2]').text
            subsubtable['place'][j] = getPlace(text)
            subsubtable['date'][j] = datetime.datetime(*getDate(text)[:6])
        return subsubtable
    except:
            #Break the captcha and goes on
            print 'error, possibly a captcha asking'
            breakCaptha(driver)
            subsubtable = thirdLevelScrap(i)
            return subsubtable

In [44]:
def secondLevelScrap(i):
    try:
        k=2015
        subtable = pd.DataFrame(np.zeros((1,4)))
        subtable.columns=['business','url','date','place']
        while k>2013:
            subsubtable = thirdLevelScrap(i);
            k = subsubtable['date'][subsubtable['date'].size-1].year
            #section to concatenate leafs of a business
            if(subtable['business'].size<2):
                subtable = subsubtable
            pieces=[subtable,subsubtable]
            subtable = pd.concat(pieces, ignore_index=True)
            nextPage = driver.find_element_by_xpath('//*[@id="todo-conteudo-resultado"]/div/div/center/div/ul/li[7]/a').get_attribute("href")
            driver.get(nextPage)
            time.sleep(5)
        return subtable
    except:
        print "An exception screw everything on second level, returning whatever you got"
        return subtable

In [47]:
def firstLevelScrap():
    try:
        table = pd.DataFrame(np.zeros((20,4)))
        table.columns=['business','url','date','place']
        for i in range(ranking['href'].size):
                    driver.get(ranking['href'][i])
                    time.sleep(5)
                    #Take and get the url "todas as reclamações"
                    key = driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div/ul/li/a').get_attribute("href")
                    driver.get(key)
                    subtable = secondLevelScrap(i)
                    #section to concatenate business
                    pieces=[table,subtable]
                    table=pd.concat(pieces, ignore_index=True)
        return table
    except:
        print "An exception screw everything on first level, returning whatever you got"
        return table

In [48]:
k=firstLevelScrap()

An exception screw everything on first level, returning whatever you got
